# Install the neo4j-driver

In [2]:
!pip install neo4j-driver

     |████████████████████████████████| 75 kB 1.4 MB/s eta 0:00:01
Using legacy 'setup.py install' for neo4j-driver, since package 'wheel' is not installed.
    Running setup.py install for neo4j-driver ... done


## Connect a Database

In [7]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://localhost:7687", 
  auth=basic_auth("neo4j", "test"))


## Create a new Pokemon with its relantionship

In [50]:
cypher_query = '''
MATCH (r:Origin {region: 'Kanto'})
MATCH (t:Type {name: 'Grass'})
CREATE (p: Pokemon {name : 'Bulbasaur'})
CREATE (p)-[:IS_FROM]->(r)
CREATE (p)-[:IS_TYPE]->(t)
return p.name as name, r.region as region, collect(t) as type
'''

with driver.session(database="neo4j") as session:
    results = session.write_transaction(lambda tx: tx.run(cypher_query).data())

    for record in results:
        print(record)

driver.close()

{'name': 'Bulbasaur', 'region': 'Kanto', 'type': [{'name': 'Grass'}]}


## Retrieve All Pokemons and its relantionship

In [52]:
cypher_query = '''
MATCH (p)-[:IS_FROM]->(r)
MATCH (p)-[:IS_TYPE]->(t)
return p.name as name, r.region as region, collect(t) as types
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query).data())

  for record in results:
    print(record)

driver.close()

{'name': 'Bulbasaur', 'region': 'Kanto', 'types': [{'name': 'Grass'}]}
{'name': 'Venusaur', 'region': 'Kanto', 'types': [{'name': 'Poison'}, {'name': 'Grass'}]}
{'name': 'Charizard', 'region': 'Kanto', 'types': [{'name': 'Fire'}, {'name': 'Flying'}]}
{'name': 'Blastoise', 'region': 'Kanto', 'types': [{'name': 'Water'}]}
{'name': 'Meganium', 'region': 'Johto', 'types': [{'name': 'Grass'}]}
{'name': 'Typhlosion', 'region': 'Johto', 'types': [{'name': 'Fire'}]}
{'name': 'Feraligatr', 'region': 'Johto', 'types': [{'name': 'Water'}]}
{'name': 'Sceptile', 'region': 'Hoenn', 'types': [{'name': 'Grass'}]}
{'name': 'Blaziken', 'region': 'Hoenn', 'types': [{'name': 'Fighting'}, {'name': 'Fire'}]}
{'name': 'Swampert', 'region': 'Hoenn', 'types': [{'name': 'Ground'}, {'name': 'Water'}]}
{'name': 'Torterra', 'region': 'Sinnoh', 'types': [{'name': 'Grass'}, {'name': 'Ground'}]}
{'name': 'Infernape', 'region': 'Sinnoh', 'types': [{'name': 'Fighting'}, {'name': 'Fire'}]}
{'name': 'Empoleon', 'region':

## Retrieve Only Bulbasaur

In [58]:
cypher_query = '''
MATCH (p: Pokemon {name: 'Bulbasaur'})-[:IS_FROM]->(r)
MATCH (p)-[:IS_TYPE]->(t)
return p.name as name, r.region as region, collect(t) as types
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query).data())

  for record in results:
    print(record)

driver.close()

## Update Bulbasaur Region

In [55]:
cypher_query = '''
MATCH (p: Pokemon {name: 'Bulbasaur'})-[r:IS_FROM]->()
DELETE r
WITH p
MATCH(region: Origin {region: 'Galar'})
CREATE (p)-[:IS_FROM]->(region)
return p.name, region.region
'''

with driver.session(database="neo4j") as session:
    results = session.write_transaction(lambda tx: tx.run(cypher_query).data())

    for record in results:
        print(record)

driver.close()

{'p.name': 'Bulbasaur', 'region.region': 'Galar'}


## Delete Bulbasaur

In [57]:
cypher_query = '''
MATCH (p: Pokemon {name: 'Bulbasaur'})
DETACH
DELETE p
'''

with driver.session(database="neo4j") as session:
    results = session.write_transaction(lambda tx: tx.run(cypher_query).data())

    for record in results:
        print(record)

driver.close()